# Proyecto 2: Análisis Exploratorio
### Tema 12: Google - American Sign Language Fingerspelling Recognition
<div align="center">
<img src=./data/image.png alt="Sign Language Fingerspelling Recognition" width="600"/>
</div>

**Integrantes:**
- Derek Fabian Arreaga Kelson | 22537
- Mónica Alejandra Salvatierra Chacón | 22249
- Sofía Mishell Velásquez Mérida | 22049
- Paula Rebeca Barillas Alvarez | 22764

## Librerías principales para análisis de datos y visualización

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

## Lectura de los datos principales

In [3]:
train = pd.read_csv('./data/train.csv')
metadata = pd.read_csv('./data/supplemental_metadata.csv')
import json
with open('./data/character_to_prediction_index.json', 'r') as f:
    char_map = json.load(f)

## Tipo de datos del dataset

In [5]:
print('Dimensiones del dataset:', train.shape)
print('\nTipos de datos:')
print(train.dtypes)
print('\nPrimeras filas:')
display(train.head())

Dimensiones del dataset: (67208, 5)

Tipos de datos:
path              object
file_id            int64
sequence_id        int64
participant_id     int64
phrase            object
dtype: object

Primeras filas:


,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


## Variables numéricas

In [8]:
# variables numéricas
train.describe().T

,count,mean,std,min,25%,50%,75%,max
file_id,67208.0,1.094448e+09,6.395616e+08,5414471.0,5.277082e+08,1.099408e+09,1.662743e+09,2.118949e+09
sequence_id,67208.0,1.072696e+09,6.177372e+08,71095.0,5.376519e+08,1.074387e+09,1.605592e+09,2.147465e+09
participant_id,67208.0,1.197582e+02,7.433047e+01,0.0,6.300000e+01,1.130000e+02,1.780000e+02,2.540000e+02


## Tablas de frecuencia para variables categóricas

In [ ]:
cat_cols = ['participant_id', 'file_id', 'sequence_id', 'phrase']
for col in cat_cols:
    print(f'\nFrecuencia de valores en {col}:')
    print(train[col].value_counts().head(10))


Frecuencia de valores en participant_id:
participant_id
36     1535
105    1006
112     953
81      944
89      896
188     895
56      890
178     886
141     886
2       885
Name: count, dtype: int64

Frecuencia de valores en file_id:
file_id
5414471      1000
105143404    1000
128822441    1000
149822653    1000
152029243    1000
234418913    1000
296317215    1000
433948159    1000
522550314    1000
527708222    1000
Name: count, dtype: int64

Frecuencia de valores en sequence_id:
sequence_id
1816796431    1
1816825349    1
1816909464    1
1816967051    1
1817123330    1
1817141095    1
1817169529    1
1817171518    1
1817195757    1
1817216847    1
Name: count, dtype: int64

Frecuencia de valores en phrase:
phrase
surprise az                        17
yonkers new york                   16
fort worth tx                      16
raleigh north carolina             15
buffalo ny                         14
north charleston south carolina    13
alexandria virginia                13
suga